In [ ]:
%pip install openai tiktoken chromadb langchain

In [2]:
import dotenv

dotenv.load_dotenv()

True

In [3]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

In [8]:
repo_path = ".."

In [9]:
loader = GenericLoader.from_filesystem(
    repo_path + "",
    glob="**/*",
    suffixes=[".cs"],
    parser=LanguageParser(language=Language.CSHARP, parser_threshold=500),
)

documents = loader.load()
len(documents)

4

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CSHARP, chunk_size=2000, chunk_overlap=200
)

texts = splitter.split_documents(documents)
len(texts)

6

In [16]:
from langchain.vectorstores import Chroma

from langchain.embeddings.openai import OpenAIEmbeddings

# from langchain.embeddings.ollama import OllamaEmbeddings

db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
# db = Chroma.from_documents(texts, OllamaEmbeddings(model="codellama:13b"))  # type: ignore
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 8})

In [19]:
from langchain.chat_models import ChatOpenAI

# from langchain.chat_models import ChatOllama
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo")  # type: ignore
# llm = ChatOllama(model="codellama:13b")  # type: ignore
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [20]:
question = "What is the YouTube API being used for?"

result = qa(question)
result["answer"]

Number of requested results 20 is greater than number of elements in index 12, updating n_results = 12


'The YouTube API allows developers to interact with the YouTube platform and access its features programmatically. It provides methods to retrieve information about videos, channels, playlists, and comments, as well as to upload, update, and delete videos. Developers can use the YouTube API to integrate YouTube functionality into their own applications, create custom YouTube experiences, and automate tasks related to managing YouTube content.'